In [16]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn import *
from sklearn import svm
from sklearn.ensemble import RandomForestRegressor
import random
from sklearn.ensemble import ExtraTreesRegressor

In [17]:
train= pd.read_csv("C:/Users/Rajat/Desktop/TookiTaki/My work/Kaggle/Restaurant Revenue Prediction/train.csv/Revenue Prediction train.csv")
test= pd.read_csv("C:/Users/Rajat/Desktop/TookiTaki/My work/Kaggle/Restaurant Revenue Prediction/test.csv/test.csv")
sample_submission=pd.read_csv("C:/Users/Rajat/Desktop/TookiTaki/My work/Kaggle/Restaurant Revenue Prediction/sampleSubmission.csv")

In [18]:
train.rename(columns={'Open Date': 'Open_Date'},inplace= True )
test.rename(columns={'Open Date': 'Open_Date'},inplace= True)

train.Open_Date=pd.to_datetime(train.Open_Date)
test.Open_Date=pd.to_datetime(test.Open_Date)

pd.to_datetime('20150101', format='%Y%m%d')

Timestamp('2015-01-01 00:00:00')

In [19]:
train['Open_Days']=pd.to_datetime('20150101', format='%Y%m%d')-train.Open_Date
train['Open_Days']=(train.Open_Days/np.timedelta64(1,'D')).astype(int)

test['Open_Days']=pd.to_datetime('20150101', format='%Y%m%d')-test.Open_Date
test['Open_Days']=(test.Open_Days/np.timedelta64(1,'D')).astype(int)

train['Open_month']=train.Open_Date.dt.month
train['Open_year']= train.Open_Date.dt.year
train['revenue']= np.sqrt(np.log(train['revenue']))
train.head()

,Id,Open_Date,City,City Group,Type,P1,P2,P3,P4,P5,...,P32,P33,P34,P35,P36,P37,revenue,Open_Days,Open_month,Open_year
0,0,1999-07-17,İstanbul,Big Cities,IL,4,5.0,4.0,4.0,2,...,4,5,5,4,3,4,3.943074,5647,7,1999
1,1,2008-02-14,Ankara,Big Cities,FC,4,5.0,4.0,4.0,1,...,0,0,0,0,0,0,3.968675,2513,2,2008
2,2,2013-03-09,Diyarbakır,Other,IL,2,4.0,2.0,5.0,2,...,0,0,0,0,0,0,3.812607,663,3,2013
3,3,2012-02-02,Tokat,Other,IL,6,4.5,6.0,6.0,4,...,10,6,18,12,12,6,3.847031,1064,2,2012
4,4,2009-05-09,Gaziantep,Other,IL,3,4.0,3.0,4.0,2,...,3,2,3,4,3,3,3.908709,2063,5,2009


In [20]:
test['Open_month']=test.Open_Date.dt.month
test['Open_year']=test.Open_Date.dt.year
test['Type']=test['Type'].replace(['MB'], 'DT')
test.head()

,Id,Open_Date,City,City Group,Type,P1,P2,P3,P4,P5,...,P31,P32,P33,P34,P35,P36,P37,Open_Days,Open_month,Open_year
0,0,2011-01-22,Niğde,Other,FC,1,4.0,4.0,4.0,1,...,0,0,0,0,0,0,0,1440,1,2011
1,1,2011-03-18,Konya,Other,IL,3,4.0,4.0,4.0,2,...,0,0,0,0,0,0,0,1385,3,2011
2,2,2013-10-30,Ankara,Big Cities,FC,3,4.0,4.0,4.0,2,...,0,0,0,0,0,0,0,428,10,2013
3,3,2013-05-06,Kocaeli,Other,IL,2,4.0,4.0,4.0,2,...,4,0,0,0,0,0,0,605,5,2013
4,4,2013-07-31,Afyonkarahisar,Other,FC,2,4.0,4.0,4.0,1,...,0,0,0,0,0,0,0,519,7,2013


In [21]:
train['Difference']='Train' 
test['Difference']='Test'
fullData = pd.concat([train,test],axis=0)

from sklearn import preprocessing
le_Type = preprocessing.LabelEncoder()
fullData.Type= le_Type.fit_transform(fullData.Type)
fullData.City=le_Type.fit_transform(fullData.City)
fullData.head()

,City,City Group,Difference,Id,Open_Date,Open_Days,Open_month,Open_year,P1,P10,...,P36,P37,P4,P5,P6,P7,P8,P9,Type,revenue
0,60,Big Cities,Train,0,1999-07-17,5647,7,1999,4,5,...,3,4,4.0,2,2,5,4,5,2,3.943074
1,4,Big Cities,Train,1,2008-02-14,2513,2,2008,4,5,...,0,0,4.0,1,2,5,5,5,1,3.968675
2,14,Other,Train,2,2013-03-09,663,3,2013,2,5,...,0,0,5.0,2,3,5,5,5,2,3.812607
3,52,Other,Train,3,2012-02-02,1064,2,2012,6,10,...,12,6,6.0,4,4,10,8,10,2,3.847031
4,21,Other,Train,4,2009-05-09,2063,5,2009,3,5,...,3,3,4.0,2,2,5,5,5,2,3.908709


In [22]:
train=fullData[fullData['Difference']=='Train']
test=fullData[fullData['Difference']=='Test']

ID_col=['Id']
target_col=['revenue']
cat_col=['City Group','Open_Date','Difference']
num_cols=list(set(list(fullData.columns))-set(ID_col)-set(target_col)-set(cat_col))

Id_dummy=test.Id
Id_new=train.Id

features=list(set(list(fullData.columns))-set(ID_col)-set(target_col)-set(cat_col))

In [2]:
#cv=[]
#lst=[]
#kf=cross_validation.KFold(len(train),n_folds=5,random_state=0)
#for idx1,idx2 in kf:
    #x_train,x_cv=train[features].iloc[idx1],train[features].iloc[idx2]
#y_train,y_cv=train.revenue.iloc[idx1],train.revenue.iloc[idx2]
    
#model = svm.SVR(kernel='rbf', C=1, gamma=1) 
#.fit(x_train, y_train)
#.score(x_train, y_train)

In [32]:
cv=[]
lst=[]
kf=cross_validation.KFold(len(train),n_folds=5,random_state=0)
for idx1,idx2 in kf:
    x_train,x_cv=train[features].iloc[idx1],train[features].iloc[idx2]
    y_train,y_cv=train.revenue.iloc[idx1],train.revenue.iloc[idx2]
    model = ExtraTreesRegressor(n_estimators=500, criterion='mse', max_depth=20, min_samples_split=2, min_samples_leaf=2, min_weight_fraction_leaf=0.0, max_features= 'auto', max_leaf_nodes=2, bootstrap=False, oob_score=False, n_jobs=1, random_state=None, verbose=0, warm_start=False)
    mdl= model.fit(x_train,y_train)
    cv.extend(model.predict(x_cv))
    lst.append(model.predict(test[features]))
    print(metrics.mean_squared_error(mdl.predict(x_cv),y_cv))


0.00489400897757
0.00238094075839
0.0033976249786
0.00367233994429
0.00306414100997


In [29]:
df1=pd.DataFrame({'Id': Id_dummy, 'revenue':np.average(lst,axis=0)})
df2=pd.DataFrame({'Id': Id_new,'revenue': cv})

In [25]:
#df1.revenue=np.exp(np.power(df1.revenue,2))
#.revenue=np.exp(np.power(df2.revenue,2))

In [30]:
#df3=pd.DataFrame({'Id': Id_dummy, 'revenue':df1.revenue})
df1.to_csv("C:/Users/Rajat/Desktop/ExtraTressRegressor.csv",index= False)

In [31]:
#df4=pd.DataFrame({'Id': Id_new, 'revenue':df2.revenue})
df2.to_csv("C:/Users/Rajat/Desktop/ExtraTreesRegressorCV.csv",index= False)